In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv('etl_df', low_memory= False)
df.head()

,original_language,overview,popularity,release_date,status,tagline,title,vote_average,anio,return_(%),runtime_(minutos),budget_(dolares),revenue_(dolares),director,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,Released,NaN,Toy Story,7.7,1995.0,1245.0,81.0,30000000.0,373554033.0,John Lasseter,Toy Story Collection,Animation - Comedy - Family,Pixar Animation Studios,United Statesof America,English
1,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995.0,404.0,104.0,65000000.0,262797249.0,Joe Johnston,NaN,Adventure - Fantasy - Family,Tri Star Pictures - Teitler Film - Interscop...,United Statesof America,English - Français
2,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995.0,0.0,101.0,0.0,0.0,Howard Deutch,Grumpy Old Men Collection,Romance - Comedy,Warner Bros. - Lancaster Gate,United Statesof America,English
3,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,1995.0,509.0,127.0,16000000.0,81452156.0,Forest Whitaker,NaN,Comedy - Drama - Romance,Twentieth Century Fox Film Corporation,United Statesof America,English
4,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,1995.0,0.0,106.0,0.0,76578911.0,Charles Shyer,Fatherofthe Bride Collection,Comedy,Sandollar Productions - Touchstone Pictures,United Statesof America,English


In [51]:
def peliculas_idioma(idioma: str):
    cont = df['original_language'].value_counts()[idioma]
    return print(cont, 'peliculas se estrenaron en', idioma)

In [52]:
peliculas_idioma('es')

992 peliculas se estrenaron en es


In [53]:
def peliculas_duracion(titulo: str):
    dato = titulo.title()
    pelicula = df.loc[df.index[df['title'] == dato].tolist(), 'title'].iloc[0]
    duracion = df.loc[df.index[df['title'] == dato].tolist(), 'runtime_(minutos)'].iloc[0]
    anio = df.loc[df.index[df['title'] == dato].tolist(), 'anio'].iloc[0]
    return pelicula, duracion, anio

In [54]:
peliculas_duracion('JUMANJI')

('Jumanji', 104.0, 1995.0)

In [55]:
def franquicia(franquicia:str):
    dato = franquicia.title()
    df["belongs_to_collection"] = df["belongs_to_collection"].fillna('')
    franq = df[df["belongs_to_collection"].str.contains(dato, case=False)]
    cantidad = franq['belongs_to_collection'].count()
    ganancia_total = round(franq['revenue_(dolares)'].sum() - franq['budget_(dolares)'].sum())
    ganancia_promedio = round(franq['revenue_(dolares)'].mean() - franq['budget_(dolares)'].mean())
    return dato, cantidad, ganancia_total, ganancia_promedio

    #return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

In [56]:
franquicia('Toy Story')

('Toy Story', 3, 1617890605, 539296868)

In [57]:
def peliculas_pais(pais:str):
    dato = pais.title()
    df["production_countries"] = df["production_countries"].fillna('')
    countr = df[df["production_countries"].str.contains(dato, case=False)]
    respuesta = countr['production_countries'].count()

    return dato, respuesta
    #return {'pais':pais, 'cantidad':respuesta}


In [58]:
peliculas_pais('argentina')

('Argentina', 254)

In [59]:
def productoras_exitosas(productora:str):
    dato = productora.title()
    df["production_companies"] = df["production_companies"].fillna('')
    prod = df[df["production_companies"].str.contains(dato, case=False)]
    revenue_total = round(prod['revenue_(dolares)'].sum())
    cantidad = prod['title'].count()

    return dato, revenue_total, cantidad
    # return {'productora':productora, 'revenue_total': respuesta,'cantidad':respuesta}

In [60]:
productoras_exitosas('Warner Bros')

('Warner Bros', 67083668593, 1372)

In [61]:
def get_director(nombre_director:str):
    dato = nombre_director.title()
    dir = df[df['director'] == dato]
    retorno_total_director = dir['revenue_(dolares)'].sum()
    peliculas = dir['title'].unique()
    anio = dir['anio']
    retorno_pelicula = dir['return_(%)']
    budget_pelicula = dir['budget_(dolares)']
    revenue_pelicula = dir['revenue_(dolares)']
    if dir.empty:
       error = {'error': "f'{dato} nombre incorrecto"}
       return error

    return [dato, retorno_total_director, peliculas, anio, retorno_pelicula, budget_pelicula, revenue_pelicula]
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma. En formato lista'''
    
    # return {'director':nombre_director, 'retorno_total_director':respuesta, 
    #'peliculas':respuesta, 'anio':respuesta,, 'retorno_pelicula':respuesta, 
    #'budget_pelicula':respuesta, 'revenue_pelicula':respuesta}

In [62]:
get_director('John Lasseter')

['John Lasseter',
 2256015306.0,
 array(['Toy Story', "A Bug's Life", 'Toy Story 2', 'Luxo Jr.', 'Cars',
        'Cars 2', 'Tin Toy', "Red's Dream", 'Knick Knack',
        'Mater and the Ghostlight'], dtype=object),
 0        1995.0
 2248     1998.0
 3006     1999.0
 10693    1986.0
 11013    2006.0
 17423    2011.0
 19169    1988.0
 19223    1987.0
 19273    1989.0
 22901    2006.0
 Name: anio, dtype: float64,
 0        1245.0
 2248      303.0
 3006      553.0
 10693       0.0
 11013     385.0
 17423     280.0
 19169       0.0
 19223       0.0
 19273       0.0
 22901       0.0
 Name: return_(%), dtype: float64,
 0         30000000.0
 2248     120000000.0
 3006      90000000.0
 10693            0.0
 11013    120000000.0
 17423    200000000.0
 19169            0.0
 19223            0.0
 19273            0.0
 22901            0.0
 Name: budget_(dolares), dtype: float64,
 0        373554033.0
 2248     363258859.0
 3006     497366869.0
 10693            0.0
 11013    461983149.0
 17423   